In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy import linalg as LA

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [3]:
# 正弦波をノイズありで生成する

# 学習データ
learningX = np.sort(np.random.random(size=25)*5)
learningY = np.sin(learningX)+np.random.normal(scale=0.1, size=25)
# テストデータ
testX = np.sort(np.random.random(size=25)*5)
testY = np.sin(testX)+np.random.normal(scale=0.1, size=25)

# 真値
trueX = np.arange(0, 5, 0.1)
trueY = np.sin(trueX)

In [22]:
# CoreMLがscikit-learnの基底関数をサポートしないので，手で4次の基底関数を作る
x1 = learningX;
x = np.transpose([x1, x1*x1, x1*x1*x1, x1*x1*x1*x1]);
# y_inference = lreg.predict(x)#x.dot(lreg.coef_);
# yt = regr.predict(x1.reshape(-1,1))
reg = linear_model.LinearRegression()
reg.fit(x, learningY)

print(reg.coef_)

print(reg)

# 学習したパラメータの学習性能を評価する
inferenceY = reg.predict(x)
error2Norm = LA.norm(learningY - inferenceY, 2)
print(error2Norm)

# 学習したパラメータの汎化性能を評価する
x1 = testX;
x = np.transpose([x1, x1*x1, x1*x1*x1, x1*x1*x1*x1]);
inferenceY = reg.predict(x)
error2Norm = LA.norm(testY - inferenceY, 2)
print(error2Norm)

[ 1.28478795 -0.26969847 -0.10784114  0.02085694]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
0.439893033455
0.490154754603


In [28]:
# CoreML用にモデルとパラメータを書き出す
from coremltools.converters import sklearn  
coreml_model = sklearn.convert(reg, ["x1", "x2", "x3", "x4"])
coreml_model.author = 'Yuichi Yoshida'
coreml_model.license = 'BSD'
coreml_model.short_description = 'CoreML Test'
coreml_model.save('./output/linear_model.mlmodel')

In [29]:
# CoreML用にモデルとパラメータを書き出す
from coremltools.converters import sklearn  
coreml_model = sklearn.convert(reg, ['x1', "x2", "x3", "x4"])
coreml_model.author = 'Yuichi Yoshida'
coreml_model.license = 'MIT'
coreml_model.short_description = 'CoreML Test'
coreml_model.input_description['x1'] = u'1次の入力値．推定したい値を入力してください．'
coreml_model.input_description['x2'] = u'2次の入力値．推定したい値の2乗を入力してください．'
coreml_model.input_description['x3'] = u'3次の入力値．推定したい値を3乗を入力してください．'
coreml_model.input_description['x4'] = u'4次の入力値．推定したい値を4乗を入力してください．'
coreml_model.output_description['prediction'] = u'推定した結果'
coreml_model.save('./output/linear_model.mlmodel')